%md
# 🔄 Silver Layer - Transformação e Limpeza de Dados
 
### **Objetivo**: Transformar dados brutos em dados limpos e padronizados 
### 📊 Processos aplicados:
### **Limpeza**: Remoção de duplicatas, tratamento de nulos
### **Padronização**: Normalização de formatos e tipos
### **Enriquecimento**: Adição de informações derivadas
### **Validação**: Regras de qualidade de dados
### **Documentação**: Catálogo de dados

## 1️⃣ Setup e Configuração

In [0]:
# Importar bibliotecas
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime
import time

print("✅ Bibliotecas importadas!")


In [0]:
# Configurar catalog e schemas
catalog_name = "datasus_project"
schema_bronze = "bronze"
schema_silver = "silver"

spark.sql(f"USE CATALOG {catalog_name}")

print(f"✅ Usando catalog: {catalog_name}")

## 2️⃣ Ler Dados da Bronze Layer


In [0]:
# Ler tabela CNES
df_bronze = spark.table(f"{catalog_name}.{schema_bronze}.cnes_estabelecimentos_raw")

print("✅ Dados CNES carregados!")
print(f"\n📊 DATASET BRONZE:")
print(f"   Registros: {df_bronze.count():,}")
print(f"   Colunas: {len(df_bronze.columns)}")

# View Temporária para performance
df_bronze.createOrReplaceTempView("bronze_cnes_estabelecimentos")

In [0]:
# Ver amostra
print("\n📋 Amostra dos dados:")
df_bronze.select(
    "CO_CNES", "NO_FANTASIA", "CO_UF", "NU_LATITUDE", "NU_LONGITUDE",
    "TP_UNIDADE", "CO_ATIVIDADE", "CO_NATUREZA_ORGANIZACAO"
).show(5, truncate=True)

## 3️⃣ Análise Exploratória de Qualidade

In [0]:
print("🔍 ANÁLISE DE QUALIDADE - CNES")
print("="*70)

total_records = df_bronze.count()
print(f"\n📊 Total de estabelecimentos: {total_records:,}")

In [0]:
# Verificar duplicatas por CO_CNES
duplicates = df_bronze.groupBy("CO_CNES").count().filter(F.col("count") > 1).count()
print(f"\n🔍 Duplicatas por CO_CNES: {duplicates}")

if duplicates > 0:
    print("   ⚠️ Encontradas duplicatas - serão removidas")


In [0]:
# Análise de valores nulos
print("\n📉 VALORES NULOS - CAMPOS PRINCIPAIS:")
print("="*70)

key_columns = [
    "CO_CNES", "NO_FANTASIA", "CO_UF", "NU_LATITUDE", "NU_LONGITUDE",
    "TP_UNIDADE", "CO_ATIVIDADE", "NU_CNPJ", "NO_RAZAO_SOCIAL"
]

for col in key_columns:
    if col in df_bronze.columns:
        null_count = df_bronze.filter(F.col(col).isNull()).count()
        percentage = (null_count / total_records) * 100
        status = "⚠️" if percentage > 10 else "✅"
        print(f"{status} {col:30s}: {null_count:>8,} ({percentage:>5.2f}%)")


In [0]:
# Distribuição por UF
print("\n📍 ESTABELECIMENTOS POR ESTADO:")
df_bronze.groupBy("CO_UF") \
    .agg(F.count("*").alias("total_estabelecimentos")) \
    .orderBy(F.desc("total_estabelecimentos")) \
    .show(27)  # 27 UFs do Brasil

## 4️⃣ Transformações - Limpeza de Dados

In [0]:

print("🔄 INICIANDO TRANSFORMAÇÕES SILVER")
print("="*70)

df_silver = df_bronze

In [0]:
# 1. Remover duplicatas (manter apenas o primeiro registro por CNES)
if duplicates > 0:
    records_before = df_silver.count()
    
    window_spec = Window.partitionBy("CO_CNES").orderBy(F.col("data_ingestao").desc())
    
    df_silver = df_silver.withColumn("row_num", F.row_number().over(window_spec)) \
                         .filter(F.col("row_num") == 1) \
                         .drop("row_num")
    
    records_after = df_silver.count()
    removed = records_before - records_after
    
    print(f"✅ Duplicatas removidas: {removed:,}")
    print(f"   Antes: {records_before:,} | Depois: {records_after:,}")
else:
    print("✅ Nenhuma duplicata encontrada")

In [0]:
# 2. Limpar e padronizar nomes
print("\n🔤 Padronizando textos...")

# Remover espaços extras e padronizar
text_columns = ["NO_FANTASIA", "NO_RAZAO_SOCIAL", "NO_LOGRADOURO", "NO_BAIRRO"]

for col in text_columns:
    if col in df_silver.columns:
        df_silver = df_silver.withColumn(
            f"{col}_LIMPO",
            F.trim(F.upper(F.col(col)))
        )

print("✅ Textos padronizados (UPPERCASE, sem espaços extras)")

In [0]:
# 3. Converter códigos numéricos
print("\n🔢 Convertendo tipos de dados...")

numeric_conversions = {
    "CO_CNES": "bigint",
    "CO_UF": "int",
    "CO_IBGE": "bigint",
    "CO_UNIDADE": "bigint",
    "TP_UNIDADE": "int",
    "CO_CEP": "bigint",
    "CO_ATIVIDADE": "int",
    "NU_CNPJ": "double",
    "CO_TURNO_ATENDIMENTO": "double",
    "CO_NIVEL_HIERARQUIA": "double"
}

for col, dtype in numeric_conversions.items():
    if col in df_silver.columns:
        df_silver = df_silver.withColumn(
            f"{col}_CONV",
            F.col(col).cast(dtype)
        )

print("✅ Conversões de tipo aplicadas")

## 5️⃣ Enriquecimento - Criação de Campos Derivados

In [0]:
print("\n🌟 CRIANDO CAMPOS DERIVADOS")
print("="*70)


# 1. Adicionar nome do estado e região
print("\n📍 Adicionando informações geográficas...")

# Dicionário de UFs
ufs_brasil = {
    11: ('RO', 'Rondônia', 'Norte'),
    12: ('AC', 'Acre', 'Norte'),
    13: ('AM', 'Amazonas', 'Norte'),
    14: ('RR', 'Roraima', 'Norte'),
    15: ('PA', 'Pará', 'Norte'),
    16: ('AP', 'Amapá', 'Norte'),
    17: ('TO', 'Tocantins', 'Norte'),
    21: ('MA', 'Maranhão', 'Nordeste'),
    22: ('PI', 'Piauí', 'Nordeste'),
    23: ('CE', 'Ceará', 'Nordeste'),
    24: ('RN', 'Rio Grande do Norte', 'Nordeste'),
    25: ('PB', 'Paraíba', 'Nordeste'),
    26: ('PE', 'Pernambuco', 'Nordeste'),
    27: ('AL', 'Alagoas', 'Nordeste'),
    28: ('SE', 'Sergipe', 'Nordeste'),
    29: ('BA', 'Bahia', 'Nordeste'),
    31: ('MG', 'Minas Gerais', 'Sudeste'),
    32: ('ES', 'Espírito Santo', 'Sudeste'),
    33: ('RJ', 'Rio de Janeiro', 'Sudeste'),
    35: ('SP', 'São Paulo', 'Sudeste'),
    41: ('PR', 'Paraná', 'Sul'),
    42: ('SC', 'Santa Catarina', 'Sul'),
    43: ('RS', 'Rio Grande do Sul', 'Sul'),
    50: ('MS', 'Mato Grosso do Sul', 'Centro-Oeste'),
    51: ('MT', 'Mato Grosso', 'Centro-Oeste'),
    52: ('GO', 'Goiás', 'Centro-Oeste'),
    53: ('DF', 'Distrito Federal', 'Centro-Oeste'),
}

# Criar arrays para mapping
uf_codes = [k for k in ufs_brasil.keys()]
uf_siglas = [v[0] for v in ufs_brasil.values()]
uf_nomes = [v[1] for v in ufs_brasil.values()]
uf_regioes = [v[2] for v in ufs_brasil.values()]

# Criar maps
mapping_sigla = F.create_map([F.lit(x) for pair in zip(uf_codes, uf_siglas) for x in pair])
mapping_nome = F.create_map([F.lit(x) for pair in zip(uf_codes, uf_nomes) for x in pair])
mapping_regiao = F.create_map([F.lit(x) for pair in zip(uf_codes, uf_regioes) for x in pair])

# Aplicar
df_silver = df_silver \
    .withColumn("UF_SIGLA", mapping_sigla[F.col("CO_UF")]) \
    .withColumn("UF_NOME", mapping_nome[F.col("CO_UF")]) \
    .withColumn("REGIAO", mapping_regiao[F.col("CO_UF")])

print("✅ Criado: UF_SIGLA, UF_NOME, REGIAO")

In [0]:
# 2. Classificar tipo de estabelecimento
print("\n🏥 Classificando estabelecimentos...")

# Baseado no TP_UNIDADE (valores comuns do CNES)
df_silver = df_silver.withColumn("TIPO_ESTABELECIMENTO",
    F.when(F.col("TP_UNIDADE") == 1, "POSTO DE SAÚDE")
     .when(F.col("TP_UNIDADE") == 2, "CENTRO DE SAÚDE")
     .when(F.col("TP_UNIDADE") == 4, "POLICLÍNICA")
     .when(F.col("TP_UNIDADE") == 5, "HOSPITAL GERAL")
     .when(F.col("TP_UNIDADE") == 7, "HOSPITAL ESPECIALIZADO")
     .when(F.col("TP_UNIDADE") == 15, "UNIDADE MISTA")
     .when(F.col("TP_UNIDADE") == 20, "PRONTO SOCORRO GERAL")
     .when(F.col("TP_UNIDADE") == 21, "PRONTO SOCORRO ESPECIALIZADO")
     .when(F.col("TP_UNIDADE") == 36, "CLÍNICA ESPECIALIZADA")
     .when(F.col("TP_UNIDADE") == 39, "UNIDADE DE APOIO DIAGNOSE E TERAPIA")
     .when(F.col("TP_UNIDADE") == 40, "UNIDADE MÓVEL FLUVIAL")
     .when(F.col("TP_UNIDADE") == 42, "UNIDADE MÓVEL TERRESTRE")
     .when(F.col("TP_UNIDADE") == 61, "CENTRO DE PARTO NORMAL")
     .when(F.col("TP_UNIDADE") == 62, "HOSPITAL/DIA")
     .when(F.col("TP_UNIDADE") == 69, "CENTRO DE ATENÇÃO HEMOTERAPIA")
     .when(F.col("TP_UNIDADE") == 70, "CENTRO DE ATENÇÃO PSICOSSOCIAL")
     .when(F.col("TP_UNIDADE") == 71, "CENTRO DE APOIO À SAÚDE DA FAMÍLIA")
     .when(F.col("TP_UNIDADE") == 72, "UNIDADE DE ATENÇÃO À SAÚDE INDÍGENA")
     .when(F.col("TP_UNIDADE") == 73, "PRONTO ATENDIMENTO")
     .when(F.col("TP_UNIDADE") == 74, "POLO ACADEMIA DA SAÚDE")
     .when(F.col("TP_UNIDADE") == 79, "OFICINA ORTOPÉDICA")
     .when(F.col("TP_UNIDADE") == 80, "LABORATÓRIO CENTRAL DE SAÚDE PÚBLICA")
     .when(F.col("TP_UNIDADE") == 81, "CENTRAL DE GESTÃO EM SAÚDE")
     .when(F.col("TP_UNIDADE") == 82, "CENTRAL DE NOTIFICAÇÃO/VIGILÂNCIA")
     .when(F.col("TP_UNIDADE") == 83, "POLO DE PREVENÇÃO DE DOENÇAS")
     .otherwise("OUTRO ESTABELECIMENTO")
)

print("✅ Criado: TIPO_ESTABELECIMENTO")

In [0]:
# 3. Categorizar complexidade
print("\n⚕️ Categorizando complexidade...")

df_silver = df_silver.withColumn("COMPLEXIDADE",
    F.when(F.col("TP_UNIDADE").isin([1, 2, 71, 74]), "ATENÇÃO BÁSICA")
     .when(F.col("TP_UNIDADE").isin([4, 36, 39, 70, 73]), "MÉDIA COMPLEXIDADE")
     .when(F.col("TP_UNIDADE").isin([5, 7, 20, 21, 62]), "ALTA COMPLEXIDADE")
     .otherwise("NÃO CLASSIFICADO")
)

print("✅ Criado: COMPLEXIDADE")

In [0]:
# 4. Classificar natureza da organização
print("\n🏢 Classificando natureza jurídica...")

df_silver = df_silver.withColumn("NATUREZA_ORGANIZACAO_DESC",
    F.when(F.col("CO_NATUREZA_ORGANIZACAO") == 1, "ADMINISTRAÇÃO PÚBLICA")
     .when(F.col("CO_NATUREZA_ORGANIZACAO") == 2, "ENTIDADES EMPRESARIAIS")
     .when(F.col("CO_NATUREZA_ORGANIZACAO") == 3, "ENTIDADES SEM FINS LUCRATIVOS")
     .when(F.col("CO_NATUREZA_ORGANIZACAO") == 4, "PESSOAS FÍSICAS")
     .when(F.col("CO_NATUREZA_ORGANIZACAO") == 5, "ORGANIZAÇÕES INTERNACIONAIS")
     .otherwise("NÃO ESPECIFICADO")
)

print("✅ Criado: NATUREZA_ORGANIZACAO_DESC")

In [0]:
# 5. Validar coordenadas geográficas
print("\n🌍 Validando localização geográfica...")

df_silver = df_silver.withColumn("TEM_LOCALIZACAO",
    F.when(
        (F.col("NU_LATITUDE").isNotNull()) & 
        (F.col("NU_LONGITUDE").isNotNull()) &
        (F.col("NU_LATITUDE") != 0) &
        (F.col("NU_LONGITUDE") != 0),
        True
    ).otherwise(False)
)

# Validar se coordenadas estão dentro do Brasil (aproximadamente)
df_silver = df_silver.withColumn("LOCALIZACAO_VALIDA",
    F.when(
        (F.col("TEM_LOCALIZACAO") == True) &
        (F.col("NU_LATITUDE").between(-35, 6)) &  # Latitude do Brasil
        (F.col("NU_LONGITUDE").between(-75, -30)),  # Longitude do Brasil
        True
    ).otherwise(False)
)

print("✅ Criado: TEM_LOCALIZACAO, LOCALIZACAO_VALIDA")

In [0]:
# 6. Flags de serviços especializados
print("\n🏥 Criando flags de serviços...")

# Centro cirúrgico
df_silver = df_silver.withColumn("FLAG_CENTRO_CIRURGICO",
    F.when(F.col("ST_CENTRO_CIRURGICO").isNotNull() & (F.col("ST_CENTRO_CIRURGICO") > 0), True)
     .otherwise(False)
)

# Centro obstétrico
df_silver = df_silver.withColumn("FLAG_CENTRO_OBSTETRICO",
    F.when(F.col("ST_CENTRO_OBSTETRICO").isNotNull() & (F.col("ST_CENTRO_OBSTETRICO") > 0), True)
     .otherwise(False)
)

# Centro neonatal
df_silver = df_silver.withColumn("FLAG_CENTRO_NEONATAL",
    F.when(F.col("ST_CENTRO_NEONATAL").isNotNull() & (F.col("ST_CENTRO_NEONATAL") > 0), True)
     .otherwise(False)
)

# Atendimento hospitalar
df_silver = df_silver.withColumn("FLAG_ATEND_HOSPITALAR",
    F.when(F.col("ST_ATEND_HOSPITALAR").isNotNull() & (F.col("ST_ATEND_HOSPITALAR") > 0), True)
     .otherwise(False)
)

# Serviço de apoio
df_silver = df_silver.withColumn("FLAG_SERVICO_APOIO",
    F.when(F.col("ST_SERVICO_APOIO").isNotNull() & (F.col("ST_SERVICO_APOIO") > 0), True)
     .otherwise(False)
)

# Atendimento ambulatorial
df_silver = df_silver.withColumn("FLAG_ATEND_AMBULATORIAL",
    F.when(F.col("ST_ATEND_AMBULATORIAL").isNotNull() & (F.col("ST_ATEND_AMBULATORIAL") > 0), True)
     .otherwise(False)
)

print("✅ Criado: 6 flags de serviços especializados")

In [0]:
# 7. Criar score de capacidade do estabelecimento
print("\n📊 Calculando score de capacidade...")

df_silver = df_silver.withColumn("SCORE_CAPACIDADE",
    (F.when(F.col("FLAG_CENTRO_CIRURGICO") == True, 20).otherwise(0)) +
    (F.when(F.col("FLAG_CENTRO_OBSTETRICO") == True, 20).otherwise(0)) +
    (F.when(F.col("FLAG_CENTRO_NEONATAL") == True, 15).otherwise(0)) +
    (F.when(F.col("FLAG_ATEND_HOSPITALAR") == True, 25).otherwise(0)) +
    (F.when(F.col("FLAG_SERVICO_APOIO") == True, 10).otherwise(0)) +
    (F.when(F.col("FLAG_ATEND_AMBULATORIAL") == True, 10).otherwise(0))
)

df_silver = df_silver.withColumn("CATEGORIA_CAPACIDADE",
    F.when(F.col("SCORE_CAPACIDADE") >= 80, "ALTA CAPACIDADE")
     .when(F.col("SCORE_CAPACIDADE") >= 50, "MÉDIA CAPACIDADE")
     .when(F.col("SCORE_CAPACIDADE") >= 20, "BAIXA CAPACIDADE")
     .otherwise("CAPACIDADE MÍNIMA")
)

print("✅ Criado: SCORE_CAPACIDADE, CATEGORIA_CAPACIDADE")

## 6️⃣ Validação de Qualidade

In [0]:
print("\n✔️ APLICANDO VALIDAÇÕES DE QUALIDADE")
print("="*70)

# Iniciar score de qualidade
df_silver = df_silver.withColumn("score_qualidade", F.lit(100))

# Validação 1: CNES válido
df_silver = df_silver.withColumn("score_qualidade",
    F.when(F.col("CO_CNES").isNull(), F.col("score_qualidade") - 50)
     .otherwise(F.col("score_qualidade"))
)

# Validação 2: Nome do estabelecimento
df_silver = df_silver.withColumn("score_qualidade",
    F.when(
        F.col("NO_FANTASIA").isNull() | 
        (F.length(F.trim(F.col("NO_FANTASIA"))) < 3),
        F.col("score_qualidade") - 10
    ).otherwise(F.col("score_qualidade"))
)

# Validação 3: Localização
df_silver = df_silver.withColumn("score_qualidade",
    F.when(F.col("LOCALIZACAO_VALIDA") == False, F.col("score_qualidade") - 15)
     .otherwise(F.col("score_qualidade"))
)

# Validação 4: UF válida
df_silver = df_silver.withColumn("score_qualidade",
    F.when(F.col("UF_SIGLA").isNull(), F.col("score_qualidade") - 20)
     .otherwise(F.col("score_qualidade"))
)

# Validação 5: CNPJ
df_silver = df_silver.withColumn("score_qualidade",
    F.when(F.col("NU_CNPJ").isNull(), F.col("score_qualidade") - 5)
     .otherwise(F.col("score_qualidade"))
)

print("✅ Validações aplicadas: 5 regras de qualidade")

In [0]:
# Classificar qualidade geral
df_silver = df_silver.withColumn("CLASSIFICACAO_QUALIDADE",
    F.when(F.col("score_qualidade") >= 90, "EXCELENTE")
     .when(F.col("score_qualidade") >= 70, "BOA")
     .when(F.col("score_qualidade") >= 50, "REGULAR")
     .otherwise("BAIXA")
)

In [0]:
# Ver distribuição de qualidade
print("\n📊 DISTRIBUIÇÃO DE QUALIDADE:")
df_silver.groupBy("CLASSIFICACAO_QUALIDADE") \
    .count() \
    .orderBy(F.desc("count")) \
    .show()
    

## 7️⃣ Adicionar Metadados

In [0]:
# Adicionar metadados de processamento
df_silver = df_silver \
    .withColumn("data_processamento_silver", F.current_timestamp()) \
    .withColumn("versao_silver", F.lit("1.0")) \
    .withColumn("pipeline_executado", F.lit("silver_cnes_v1"))

print("✅ Metadados adicionados")

## 8️⃣ Salvar Silver Layer

In [0]:

print("\n💾 SALVANDO DADOS NA SILVER LAYER")
print("="*70)

table_silver = f"{catalog_name}.{schema_silver}.cnes_estabelecimentos_clean"

print(f"\n📌 Tabela destino: {table_silver}")

final_count = df_silver.count()
print(f"📊 Registros a salvar: {final_count:,}")

In [0]:
# Salvar com particionamento por UF e Região
start_time = time.time()

df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("REGIAO", "UF_SIGLA") \
    .saveAsTable(table_silver)

exec_time = time.time() - start_time

print(f"\n✅ Dados salvos com sucesso!")
print(f"   Tempo: {exec_time:.2f}s")
print(f"   Velocidade: {final_count/exec_time:.0f} registros/segundo")

## 9️⃣ Otimizar Tabela

In [0]:
print("\n⚡ OTIMIZANDO TABELA DELTA")
print("="*70)

# Otimizar
spark.sql(f"OPTIMIZE {table_silver}")
print("✅ OPTIMIZE executado")

# Z-Order por campos comumente filtrados
spark.sql(f"OPTIMIZE {table_silver} ZORDER BY (CO_CNES, TP_UNIDADE, CO_ATIVIDADE)")
print("✅ Z-ORDER aplicado")

In [0]:
# Estatísticas da tabela
print("\n📊 ESTATÍSTICAS DA TABELA:")
spark.sql(f"DESCRIBE DETAIL {table_silver}").select(
    "format", "numFiles", "sizeInBytes", "partitionColumns"
).show(truncate=False)

## 🔟 Validação Final e Resumo

In [0]:
# Ler dados salvos
df_validacao = spark.table(table_silver)

print("✔️ VALIDAÇÃO FINAL")
print("="*70)
print(f"✅ Registros salvos: {df_validacao.count():,}")
print(f"✅ Colunas totais: {len(df_validacao.columns)}")

In [0]:
# Estatísticas por região
print("\n📍 ESTABELECIMENTOS POR REGIÃO:")
df_validacao.groupBy("REGIAO") \
    .agg(
        F.count("*").alias("total_estabelecimentos"),
        F.sum(F.when(F.col("FLAG_ATEND_HOSPITALAR") == True, 1).otherwise(0)).alias("com_atend_hospitalar"),
        F.avg("SCORE_CAPACIDADE").alias("score_medio_capacidade")
    ) \
    .orderBy(F.desc("total_estabelecimentos")) \
    .show()


In [0]:
# Por tipo de estabelecimento
print("\n🏥 TOP 10 TIPOS DE ESTABELECIMENTO:")
df_validacao.groupBy("TIPO_ESTABELECIMENTO") \
    .count() \
    .orderBy(F.desc("count")) \
    .show(10, truncate=False)

In [0]:
# Por complexidade
print("\n⚕️ DISTRIBUIÇÃO POR COMPLEXIDADE:")
df_validacao.groupBy("COMPLEXIDADE") \
    .count() \
    .orderBy(F.desc("count")) \
    .show()


In [0]:
# Amostra final
print("\n📋 AMOSTRA DOS DADOS SILVER:")
df_validacao.select(
    "CO_CNES", "NO_FANTASIA_LIMPO", "UF_SIGLA", "REGIAO",
    "TIPO_ESTABELECIMENTO", "COMPLEXIDADE", 
    "CATEGORIA_CAPACIDADE", "CLASSIFICACAO_QUALIDADE"
).show(10, truncate=True)

In [0]:
# Registrar log
def log_pipeline_execution(pipeline_name, status, records=0, exec_time=0, error=None):
    schema_control = StructType([
        StructField("pipeline_name", StringType(), False),
        StructField("execution_date", TimestampType(), False),
        StructField("status", StringType(), False),
        StructField("records_processed", LongType(), True),
        StructField("execution_time_seconds", DoubleType(), True),
        StructField("error_message", StringType(), True)
    ])
    log_data = [(pipeline_name, datetime.now(), status, records, exec_time, error)]
    df_log = spark.createDataFrame(log_data, schema_control)
    df_log.write.format("delta").mode("append").saveAsTable(f"{catalog_name}.bronze.pipeline_control")

log_pipeline_execution("silver_transform_cnes", "SUCCESS", final_count, exec_time)
print("\n✅ Log registrado!")

## 🎉 Resumo Final

In [0]:
print("="*70)
print("SILVER LAYER - CNES - CONCLUÍDA")
print("="*70)

print("\n✅ TRANSFORMAÇÕES APLICADAS:")
print("   ✔️ Remoção de duplicatas")
print("   ✔️ Padronização de textos")
print("   ✔️ Conversão de tipos")
print("   ✔️ Enriquecimento geográfico (UF, Região)")
print("   ✔️ Classificação de estabelecimentos (20+ tipos)")
print("   ✔️ Categorização de complexidade")
print("   ✔️ Validação de localização")
print("   ✔️ Flags de serviços (6 flags)")
print("   ✔️ Score de capacidade (0-100)")
print("   ✔️ Score de qualidade (0-100)")

print("\n📊 ESTATÍSTICAS FINAIS:")
print(f"   Tabela: {table_silver}")
print(f"   Registros: {final_count:,}")
print(f"   Colunas: {len(df_validacao.columns)}")
print(f"   Particionamento: REGIAO, UF_SIGLA")
print(f"   Otimização: OPTIMIZE + Z-ORDER aplicados")

print("\n🚀 PRÓXIMO PASSO: GOLD LAYER")
print("   - Agregações por região e tipo")
print("   - KPIs de cobertura de saúde")
print("   - Análise de distribuição geográfica")
print("   - Mapas de calor de estabelecimentos")
print("   - Datasets para dashboards")

print("\n" + "="*70)